# MetroFlow Cost Optimization
"""
Mathematical Model Overview:

Sets:
- I = {0, 1, 2, 3} → Routes A, B, C, D
- J = {0, 1} → Time blocks: Rush hour, Off-peak

Parameters:
- d_ij = demand on route i at time j
- f = fare per passenger = 2.50
- D_i = one-way distance for route i
- ec = energy cost per passenger per km = 0.05
- mc = maintenance cost per car per km = 1.00
- RC = car replacement cost amortized over 250,000 km
- CC = car capacity = 100
- mt_j = max trips allowed in time block j = 10
- MCPT = max cars per train = 6
- MCIU = max total cars in use = 30
- min_demand_pct = 85% → minimum % of demand that must be satisfied

Decision Variables:
- X_ij = number of trips on route i during time block j (integer)
- C_ij = number of cars per train on route i at time block j (integer)
- served_ij = min(X_ij * C_ij * CC, d_ij)

Objective:
Maximize total profit = revenue - (energy + maintenance + replacement costs)

Constraints:
- Trips ≤ mt
- Cars ≤ MCPT
- Total cars in use ≤ MCIU * total blocks
- served_ij ≥ 85% of d_ij
"""

In [2]:
import gurobipy as gp
from gurobipy import GRB

# Index sets
Routes = ['A', 'B', 'C', 'D']
Blocks = ['Rush', 'OffPeak']
I = range(len(Routes))  # 0–3
J = range(len(Blocks))  # 0–1

# Input Data
demand = [
    [3200, 1000],  # Route A
    [2800, 900],   # Route B
    [4000, 1500],  # Route C
    [2000, 700]    # Route D
]
distance = [12, 10, 15, 8]  # one-way km per route
fare = 2.50
ec = 0.05  # energy cost per passenger
mc = 1.00  # maintenance per km
RC = 8000  # replacement per car (amortized)
CC = 100   # car capacity
mt = [10, 10]  # max trips per block
MCPT = 6   # max cars per train
MCIU = 30  # max cars in use
min_demand_pct = 0.85

# Create the optimization model
model = gp.Model("MetroFlow_Profit_Maximization")

# Decision variables
X = model.addVars(I, J, vtype=GRB.INTEGER, name="X")  # Number of trips
C = model.addVars(I, J, vtype=GRB.INTEGER, name="C")  # Cars per train

# Intermediate variables to compute passengers served per (i,j)
served = model.addVars(I, J, name="Served")
prod = model.addVars(I, J, name="Prod")  # Represents X * C * CC (passenger capacity)

# Constraints to define passenger service based on trip and car count
for i in I:
    for j in J:
        model.addConstr(prod[i,j] == X[i,j] * C[i,j] * CC, name=f"ProdConstr_{i}_{j}")
        model.addGenConstrMin(served[i,j], [prod[i,j], demand[i][j]], name=f"MinPassengers_{i}_{j}")

# Objective: Maximize profit
model.setObjective(
    gp.quicksum(
        served[i,j] * fare -  # revenue
        X[i,j] * C[i,j] * distance[i] * (ec*CC + mc + RC/250000)  # energy + maintenance + amortized replacement
        for i in I for j in J
    ),
    GRB.MAXIMIZE
)

# Constraints
for i in I:
    for j in J:
        model.addConstr(X[i,j] <= mt[j], name=f"MaxTrips_{i}_{j}")  # max 10 trips per time block
        model.addConstr(C[i,j] <= MCPT, name=f"MaxCarsTrain_{i}_{j}")  # max 6 cars per train
        model.addConstr(served[i,j] >= min_demand_pct * demand[i][j], name=f"MinDemand_{i}_{j}")  # 85% of demand

# Constraint: total number of cars in use across all routes and blocks
model.addConstr(
    gp.quicksum(X[i,j]*C[i,j] for i in I for j in J) <= MCIU * sum(mt),
    name="MaxCarsInUse"
)

# Solve model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Profit: ${model.ObjVal:,.2f}\n")
    for i in I:
        for j in J:
            x_val = X[i,j].X
            c_val = C[i,j].X
            if x_val > 0 and c_val > 0:
                print(f"Route {Routes[i]} during {Blocks[j]}: {int(x_val)} trips, {int(c_val)} cars per train")
else:
    print("No optimal solution found.")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 24.4.0 24E248)

CPU model: Intel(R) Core(TM) i7-8569U CPU @ 2.80GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 24 rows, 32 columns and 24 nonzeros
Model fingerprint: 0x11162f83
Model has 8 quadratic objective terms
Model has 9 quadratic constraints
Model has 8 simple general constraints
  8 MIN
Variable types: 16 continuous, 16 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+02]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+00, 2e+00]
  QObjective range [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 3e+03]
  QRHS range       [6e+02, 6e+02]
  GenCon const rng [7e+02, 4e+03]
Presolve removed 23 rows and 28 columns
Presolve time: 0.26s
Presolved: 5 rows, 5 columns, 14 nonzeros
Presolved model has 1 